#4.5.3 베르누이 나이브 베이즈를 활용한 스팸 메일 분류

In [2]:
import pandas as pd

# 베르누이 나이브 베이즈를 위한 라이브러리를 임포트합니다
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import BernoulliNB

# 모델의 정확도 평가를 위해 임포트합니다
from sklearn.metrics import accuracy_score

##데이터 획득

In [3]:
email_list = [
                {'email title': 'free game only today', 'spam': True},
                {'email title': 'cheapest flight deal', 'spam': True},
                {'email title': 'limited time offer only today only today', 'spam': True},
                {'email title': 'today meeting schedule', 'spam': False},
                {'email title': 'your flight schedule attached', 'spam': False},
                {'email title': 'your credit card statement', 'spam': False} ]
df = pd.DataFrame(email_list)
df

,email title,spam
0,free game only today,True
1,cheapest flight deal,True
2,limited time offer only today only today,True
3,today meeting schedule,False
4,your flight schedule attached,False
5,your credit card statement,False


##데이터 다듬기

In [4]:
# 사이킷런의 베르누이 나이브 베이즈 분류기는 숫자만 다루기 때문에 먼저 출력(레이블)을 숫자로 변환
df['label'] = df['spam'].map({True:1,False:0})
df

,email title,spam,label
0,free game only today,True,1
1,cheapest flight deal,True,1
2,limited time offer only today only today,True,1
3,today meeting schedule,False,0
4,your flight schedule attached,False,0
5,your credit card statement,False,0


In [5]:
# 학습에 사용될 데이터와 분류값 나누기
df_x = df["email title"]
df_y = df["label"]

In [6]:
# 베르누이 나이브 베이즈 분류기의 입력 데이터(이메일 제목)는 고정된 크기의 벡터이여야함.
# 입력 데이터를 모두 고정 길이로 만들기
cv = CountVectorizer(binary=True) # binary가 True이면 출현 여부(1,0)만 판별
x_traincv = cv.fit_transform(df_x)  # fit_transform -> fit(단어학습) + transform 

In [7]:
# 전체 고정 벡터 인덱스의 단어 출력 (총 17개) 확인
cv.get_feature_names()
# a, b, c, ... 순으로 정렬됨.

['attached',
 'card',
 'cheapest',
 'credit',
 'deal',
 'flight',
 'free',
 'game',
 'limited',
 'meeting',
 'offer',
 'only',
 'schedule',
 'statement',
 'time',
 'today',
 'your']

In [8]:
# 이메일 제목들의 인코딩 벡터 확인
encoded_input=x_traincv.toarray()
encoded_input

array([[0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0],
       [0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1],
       [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1]])

In [9]:
# 첫 번째 이메일 제목의 고정된 벡터에 포함된 단어 확인
cv.inverse_transform(encoded_input[0])

[array(['free', 'game', 'only', 'today'], dtype='<U9')]

##Bernoulli Naive Bayes 모델 학습

In [10]:
# 학습 데이터로 베르누이 분류기 학습
# 사이킷런의 베르누이 나이브 베이즈는 스무딩을 기본 지원함
bnb = BernoulliNB()
bnb.fit(x_traincv,df_y) # 베르누이 분류기 학습 실행

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

##테스트 데이터 다듬기

In [11]:
test_email_list = [
                {'email title': 'free flight offer', 'spam': True},
                {'email title': 'hey traveler free flight deal', 'spam': True},
                {'email title': 'limited free game offer', 'spam': True},
                {'email title': 'today flight schedule', 'spam': False},
                {'email title': 'your credit card attached', 'spam': False},
                {'email title': 'free credit card offer only today', 'spam': False} ]
test_df = pd.DataFrame(test_email_list)
test_df['label'] = test_df['spam'].map({True:1,False:0})
test_x=test_df["email title"]
test_y=test_df["label"]
x_testcv=cv.transform(test_x) # 고정 크기 벡터로 변환
# cv.fit_transform이 아님. 상단 고정 길이 벡터 작성 시 이미 fit(단어 학습)을 했기 때문

##테스트 진행

In [12]:
# 테스트 데이터에 대해 예측 실행
predictions=bnb.predict(x_testcv)
predictions

array([1, 1, 1, 0, 0, 1])

In [13]:
# 모델 예측 정확도 출력
print("accuracy : %.2f" %(accuracy_score(test_y,predictions)*100)+"%")

accuracy : 83.33%
